# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096994


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:02<00:41,  1.49s/it]

Rendering models:  10%|█         | 3/30 [00:03<00:29,  1.08s/it]

Rendering models:  13%|█▎        | 4/30 [00:03<00:21,  1.23it/s]

Rendering models:  20%|██        | 6/30 [00:03<00:14,  1.67it/s]

...

Rendering models:  87%|████████▋ | 26/30 [00:10<00:00,  5.73it/s]

Rendering models:  90%|█████████ | 27/30 [00:10<00:00,  5.59it/s]

Rendering models:  93%|█████████▎| 28/30 [00:11<00:00,  4.33it/s]

Rendering models:  97%|█████████▋| 29/30 [00:11<00:00,  4.89it/s]

not-logged-in-f68127b03437a78136a8    0.000104
Jnursssmith5263                       0.000368
Alexander00                           0.000154
ElisabethB                            0.000105
jnarayanbvg                           0.000457
PRivate_pAuLa                         0.000192
not-logged-in-a1b9f5f016740285f28d    0.000090
awright5                              0.000142
Kartoon1x                             0.000112
not-logged-in-746dafd91ba51764d0ed    0.005599
tingard                               0.000212
Flickeroo                             0.000429
databanana                            0.000106
clairedeu                             0.000101
peakscience7                          0.000074
TRISTANM18                            0.000086
not-logged-in-3ae935e3afaee843f530    0.027057
Susymuller                            0.000089
Lavadude                              0.024561
ShirleyL                              0.000090
Paigery321                            0.000235
equidad1     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())